In [125]:
### Benjamin Tollison Hw7 ###
import sympy as sp
from IPython.display import display, Math, Latex
from sympy.solvers.solveset import linsolve
from sympy import  lambdify, Matrix,sin,cos, sinh, cosh, exp, pi, symbols, integrate
import numpy as np
import matplotlib.pyplot as plt
def displayEquations(LHS,RHS):
    left = sp.latex(LHS)
    right = sp.latex(RHS)
    display(Math(left + '=' + right))
    np.set_printoptions(suppress=True)

In [134]:
# Problem 5
    # b
K = np.array([[5,-119,296,62,-744,456],
              [-119,3125,104,-644,357,204],
              [296,104,125,-500,-350,287],
              [62,-644,-500,10125,420,-600],
              [-744,357,-350,420,3645,117],
              [456,204,287,-600,117,1620]])
F_eq = np.array([10,4,2,8,26,8])
q = np.array([0,0,0,1,0,1])
print(K@q- F_eq)
    # f
T = (1/np.sqrt(34)) * np.array([[-5,3],[-3,-5]])
point_A = T @ np.array([5000,1000])
print('Sigmas A & B:')
print(point_A)
point_B = T @ np.array([-2000,1000])
print(point_B)

[ 508 -444 -215 9517  511 1012]
Sigmas A & B:
[-3772.96887314 -3429.97170285]
[2229.48160685  171.49858514]


In [127]:
E,A,I,c,s,L = symbols('E,A,I,c,s,L')
K_sp = E * Matrix([[A/L,0,0,-A/L,0,0],[0,12*I/L**3,6*I/L**2,0,-12*I/L**3,6*I/L**2],
               [0,6*I/L**2,4*I/L,0,-6*I/L**2,2*I/L],
               [-A/L,0,0,A/L,0,0],
               [0,-12*I/L**3,-6*I/L**2,0,12*I/L**3,-6*I/L**2],
               [0,6*I/L**2,2*I/L,0,-6*I/L**2,4*I/L]])
displayEquations('K',K_sp)
T_sp = Matrix([[c,s,0,0,0,0],
               [-s,c,0,0,0,0],
               [0,0,1,0,0,0],
               [0,0,0,c,s,0],
               [0,0,0,-s,c,0],
               [0,0,0,0,0,1]])
K_global_sp = T_sp.T*K_sp*T_sp
displayEquations('K',K_global_sp)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [128]:
#### Problem 7 ####
connections = [[2,1],[2,3]]
cooridnates = [[0,0],[100,100],[400,0]]
def delta_x(element):
    x = cooridnates[element[1]-1][0] - cooridnates[element[0]-1][0]
    return x
def delta_y(element):
    y = cooridnates[element[1]-1][1] - cooridnates[element[0]-1][1]
    return y
def K_local(element):
    E = 10**7
    I = (1/12) * (1.5**4)
    L = np.sqrt(delta_x(element)**2 + delta_y(element)**2)
    A = 1.5*1.5
    K_local_cooridnates = E* np.array([[A/L,0,0,-A/L,0,0],
                                       [0,12*I/L**3,6*I/L**2,0,-12*I/L**3,6*I/L**2],
                                       [0,6*I/L**2,4*I/L,0,-6*I/L**2,2*I/L],
                                       [-A/L,0,0,A/L,0,0],
                                       [0,-12*I/L**3,-6*I/L**2,0,12*I/L**3,-6*I/L**2],
                                       [0,6*I/L**2,2*I/L,0,-6*I/L**2,4*I/L]])
    return K_local_cooridnates
def get_element_dof(element):
    i = element
    dof_list = [i[0]*3-3,i[0]*3-2,i[0]*3-1,i[1]*3-3,i[1]*3-2,i[1]*3-1]
    return dof_list
def Transpose_Matrix(element):
    L = np.sqrt(delta_x(element)**2 + delta_y(element)**2)
    c = delta_x(element) / L
    s = delta_y(element) / L
    T = np.array([[c,s,0],
                  [-s,c,0],
                  [0,0,1]])
    return T
def K_global(connectivity):
    K = np.zeros((9,9))
    for i in connectivity:
        L = np.sqrt(delta_x(i)**2 + delta_y(i)**2)
        c = delta_x(i) / L
        s = delta_y(i) / L
        Transpose_Matrix = np.array([[c,s,0,0,0,0],
                                     [-s,c,0,0,0,0],
                                     [0,0,1,0,0,0],
                                     [0,0,0,c,s,0],
                                     [0,0,0,-s,c,0],
                                     [0,0,0,0,0,1]])
        K_local_global_cooridinates = np.linalg.inv(Transpose_Matrix) @ K_local(i) @ Transpose_Matrix
        for j in range(6):
            for c in range(6):
                K[get_element_dof(i)[j]][get_element_dof(i)[c]] += K_local_global_cooridinates[j][c]
    return K
F_eq = np.array([0,0,0,5000,0,0,0,0,0])
dof_to_be_zero = [0,1,2,5,6,7,8]
def global_node_displacement(connectivity,zeros_dof):
    K_global_bc = K_global(connectivity)
    for i in zeros_dof:
        K_global_bc[i][:] = 0
        K_global_bc[:][i] = 0
        K_global_bc[i][i] = 1
    q = np.linalg.inv(K_global_bc) @ F_eq
    return q
def Global_vc(connectivity,zeros_dof):
    forces = K_global(connectivity) @ global_node_displacement(connectivity,zeros_dof) - F_eq
    return forces
print('Global Displacements')
for i in global_node_displacement(connections,dof_to_be_zero):
    print('[{}]'.format(round(i,6)))
print('Global Forces')
for i in Global_vc(connections,dof_to_be_zero):
    print('[{}]'.format(round(i,6)))


Global Displacements
[-0.0]
[0.0]
[0.0]
[0.047837]
[-0.032119]
[0.0]
[0.0]
[0.0]
[0.0]
Global Forces
[-1251.092754]
[-1249.661641]
[71.555648]
[0.0]
[0.0]
[67.671852]
[-3748.907246]
[1249.661641]
[-3.883796]


In [130]:
# part c
print('Internal forces')
node_1_internal_forces = Transpose_Matrix([2,1]) @ np.array([Global_vc(connections,dof_to_be_zero)[0],
                                                                Global_vc(connections,dof_to_be_zero)[1],
                                                                Global_vc(connections,dof_to_be_zero)[2]]) *(-1)
node_2_internal_forces = Transpose_Matrix([2,1]) @ np.array([Global_vc(connections,dof_to_be_zero)[3],
                                                                Global_vc(connections,dof_to_be_zero)[4],
                                                                Global_vc(connections,dof_to_be_zero)[5]])*(-1)
node_3_internal_forces = Transpose_Matrix([2,3]) @ np.array([Global_vc(connections,dof_to_be_zero)[6],
                                                                Global_vc(connections,dof_to_be_zero)[7],
                                                                Global_vc(connections,dof_to_be_zero)[8]])*(-1)
displayEquations('node_1',node_1_internal_forces)
displayEquations('node_2',node_2_internal_forces)
displayEquations('node_3',node_3_internal_forces)

Internal forces


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [131]:
# part d
def sigmas(node):
    h = 1.5
    I = 1/12 * h**4
    sigma_top = node[0]/h**2 - node[2]*h/I
    simga_bot = node[0]/h**2 + node[2]*h/I
    simga_shear_avg = node[1]/h**2
    return sigma_top, simga_bot, simga_shear_avg
print(sigmas(node_1_internal_forces))
print(sigmas(node_2_internal_forces))
print(sigmas(node_3_internal_forces))

(-531.4912014669579, -1040.331367772735, 0.44975541516845463)
(240.61103003564656, -240.61103003564628, 1.4291330468374983e-13)
(1742.503568850243, 1770.1216750846793, -0.010917015045068024)
